In [1]:
import import_ipynb
import torch
import dataloader
import generate_mahalanobis
import regression_mahalanobis
import generate_odin
import calmetric
from models.resnet import ResNet34
from models.densenet import DenseNet3
import os
import random
import numpy as np
import csv

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True

## No Fault Injection

In [3]:
# id_datasets = ['cifar10', 'cifar100']
# ood_dataset = 'svhn'
# model_names = ['resnet34', 'densenet3']
# batch_size = 128
# magnitude = 0.0014
# temperature = 1000

# for id_dataset in id_datasets:
#     if id_dataset == 'cifar10':
#         num_classes = 10
#     elif id_dataset == 'cifar100':
#         num_classes = 100
        
#     for model_name in model_names:
#         if model_name == 'resnet34':
#             model = ResNet34(num_c=num_classes).to(device)
#         elif model_name == 'densenet3':
#             model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
#         model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
#         model.load_state_dict(torch.load(model_path, weights_only=True))
#         model.eval()
        
#         mean, std = dataloader.get_mean_std(id_dataset)    
        
#         id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
#         _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
#         ##### ODIN #####
        
#         file_path = f'./softmax_scores/{model_name}_{id_dataset}'
        
#         if not os.path.exists(file_path):
#             os.makedirs(file_path)
            
#         generate_odin.odin(model, id_testloader, out_testloader, magnitude, temperature, std, file_path)
#         calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
        
#         ##### Mahalanobis #####
            
#         file_path = f'./output/{model_name}_{id_dataset}'
        
#         if not os.path.exists(file_path):
#             os.makedirs(file_path)
        
#         # generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
#         # regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
        

## Fault Injection - ODIN

In [4]:
import fault_injector as fi

id_datasets = ['cifar10']
ood_dataset = 'svhn'
model_names = ['resnet34']
batch_size = 128
magnitude = 0.0014
temperature = 1000
num_flips = 99

flip_pos = ''

first_forward_fi = True
backward_fi = True
second_forward_fi = True

if first_forward_fi:
    flip_pos += '_ff'

if backward_fi:
    flip_pos += '_b'
    
if second_forward_fi:
    flip_pos += '_sf'    

for id_dataset in id_datasets:
    if id_dataset == 'cifar10':
        num_classes = 10
    elif id_dataset == 'cifar100':
        num_classes = 100
        
    for model_name in model_names:
        if model_name == 'resnet34':
            model = ResNet34(num_c=num_classes).to(device)
        elif model_name == 'densenet3':
            model = DenseNet3(100, num_classes, growth_rate=12).to(device)
            
        model_path = f'./pretrained/{model_name}_{id_dataset}.pth'
        model.load_state_dict(torch.load(model_path, weights_only=True))
        model.eval()
        
        mean, std = dataloader.get_mean_std(id_dataset)    
        
        id_trainloader, id_testloader = dataloader.get_imageloader(id_dataset, batch_size, mean, std)    
        _, out_testloader = dataloader.get_imageloader(ood_dataset, batch_size, mean, std)    
        
        ##### ODIN #####
        
        file_path = f'./softmax_scores/{model_name}_{id_dataset}'
        
        if not os.path.exists(file_path):
            os.makedirs(file_path)
            
        generate_odin.odin(model, id_testloader, out_testloader, magnitude, temperature, std, file_path)

        # CSV 파일 생성 및 헤더 작성 
        results_file = f'{file_path}/fi_results_{model_name}_{id_dataset}{flip_pos}.csv' 

        with open(results_file, 'w', newline='') as csvfile: 
            csv_writer = csv.writer(csvfile) 
            csv_writer.writerow(['layer_name', 'bit_position', 'auroc', 'fpr', 'detection_error', 'auroc_fi', 'fpr_fi', 'detection_error_fi'])  
        
        # 컨볼루션 레이어와 FC 레이어 추출
        layer_names = fi.get_layer_name(model)
        
        # 각 컨볼루션 레이어 테스트 
        for layer_idx, layer_name in enumerate(layer_names): 
            print(f"Testing layer {layer_idx+1}/{len(layer_names)}: {layer_name}")  
            
            layer_results = []  
            
            # 각 비트 위치별 테스트
            for bit_position in range(0, 31):  # 주요 비트 위치만 테스트 
                print(f"  Testing bit position: {bit_position}")  
                    
                generate_odin.odin_fi(model, id_testloader, out_testloader, magnitude, temperature, std, file_path, 
                                      first_forward_fi=first_forward_fi, backward_fi=backward_fi, second_forward_fi=second_forward_fi,
                                      bit_position=bit_position, num_flips=num_flips, layer_name=layer_name)
                
                auroc, fpr, error, fi_auroc, fi_fpr, fi_error = calmetric.metric(model_name, id_dataset, ood_dataset, file_path)
                
                # 결과를 CSV에 저장 
                with open(results_file, 'a', newline='') as csvfile: 
                    csv_writer = csv.writer(csvfile) 
                    csv_writer.writerow([layer_name, bit_position, auroc, fpr, error, fi_auroc, fi_fpr, fi_error])
        
        # ##### Mahalanobis #####
            
        # file_path = f'./output/{model_name}_{id_dataset}'
        
        # if not os.path.exists(file_path):
        #     os.makedirs(file_path)
        
        # generate_mahalanobis.mahalanobis(model, id_trainloader, id_testloader, out_testloader, num_classes, magnitude, std, file_path)
        # regression_mahalanobis.regression(id_dataset, ood_dataset, file_path, score=f'Mahalanobis_{str(magnitude)}')
        

num_samples: 10000
Processing in-distribution images


KeyboardInterrupt: 